In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [2]:
using OperationsResearchModels

In [3]:
begin
    A = PertActivity("A", 1, 3, 4, [])
    B = PertActivity("B", 2, 3, 3, [])
    C = PertActivity("C", 4, 5, 5, [A, B])
    D = PertActivity("D", 3, 3, 5, [B])
    E = PertActivity("E", 2, 2, 3, [C, D])
    F = PertActivity("F", 4, 4, 4, [B, C])
    G = PertActivity("G", 2, 4, 6, [A, D])
    H = PertActivity("H", 1, 5, 6, [B, C, D])
    I = PertActivity("I", 2, 4, 5, [F, G, H])
    J = PertActivity("J", 5, 5, 6, [A, B, D, H])
    K = PertActivity("K", 1, 2, 4, [A, G, I])
end

PertActivity("K", 1.0, 2.0, 4.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("G", 2.0, 4.0, 6.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("D", 3.0, 3.0, 5.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("I", 2.0, 4.0, 5.0, PertActivity[PertActivity("F", 4.0, 4.0, 4.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[]), PertActivity("C", 4.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("G", 2.0, 4.0, 6.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("D", 3.0, 3.0, 5.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[])])]), PertActivity("H", 1.0, 5.0, 6.0, PertActivity[PertActivity("B", 2.0, 3.0, 3.0, PertActivity[]), PertActivity("C", 4.0, 5.0, 5.0, PertActivity[PertActivity("A", 1.0, 3.0, 4.0, PertActivity[]), PertActivity("B", 2.0, 3.0, 

In [5]:
problem = PertProblem([A, B, C, D, E, F, G, H, I, J, K]);

In [7]:
result = solve(problem);

In [16]:
map(x->x.name, result.path)

5-element Vector{String}:
 "A"
 "C"
 "H"
 "I"
 "K"

In [18]:
result.mean

18.166666666666668

In [19]:
result.stddev

1.2133516482134197

In [24]:
pertmean(x::PertActivity) = (x.optimistic + 4 * x.mostlikely + x.pessimistic)/6    

pertmean (generic function with 1 method)

In [26]:
pertmean(A) + pertmean(C) + pertmean(H) + pertmean(I) + pertmean(K)

18.166666666666668

In [30]:
# 95% confidence interval of average earliest completion time

In [29]:
(result.mean - 1.96 * result.stddev, result.mean + 1.96 * result.stddev)

(15.788497436168365, 20.54483589716497)